In [ ]:
from transformers import pipeline

# Load the sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")

# Analyze text
result = sentiment_analyzer("I’m concerned about the display quality. I’ll look for a better option.")
print(result)


In [2]:
pip install --upgrade transformers

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.7 MB 4.2 MB/s eta 0:00:03
   ------- -------------------------------- 1.8/9.7 MB 4.2 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/9.7 MB 4.2 MB/s eta 0:00:02
   -------------- ------------------------- 3.4/9.7 MB 4.2 MB/s eta 0:00:02
   ------------------ --------------------- 4.5/9.7 MB 4.2 MB/s eta 0:00:02
   -------------------- ------------------- 5.0/9.7 MB 4.0 MB/s eta 0:00:02
   ----------------------- ---------------- 5.8/9.7 MB 3.9 MB/s eta 0:00:02
   --------------------------- ------------ 6.6/9.7 MB 3.9 MB/s eta 0:00:01
   ------------------------------- -------- 7.6/9.7 MB 3.9 MB/s eta 0:00:01
   ---------------------------------- ----- 8.4/9.7 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------  9.4/9.7 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------  9.4/9.7 MB 4.0 MB/s eta 0:00:01
   ----------------

In [ ]:
from transformers import pipeline

# Load the summarizer pipeline with BART model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Input text (customer's query)
text = """
1. Hello, I am interested in touch screen laptops. Do you have any recommendations?
2. What's the screen resolution on the Microsoft Surface laptop 5?
3. Are there any trade-in offers if I exchange my old laptop?
4. Does the stylus come with the laptop, or is it separate?
5. This seems great, but I will check the style options first.
"""

# Preprocess the input to encourage the model to output in the desired conversational structure
formatted_text = """
Customer Inquiry Summary:
1. Customer is asking for touch screen laptop recommendations.
2. Customer wants to know the screen resolution of the Microsoft Surface Laptop 5.
3. Customer is asking if there are any trade-in offers for exchanging an old laptop.
4. Customer is asking if the stylus comes with the laptop or is purchased separately.
5. Customer mentions they will check other style options before deciding.
"""

# Generate the summary
summary = summarizer(formatted_text, max_length=200, min_length=50, do_sample=False)

# Print the summary
print("Generated Summary:")
print(summary[0]['summary_text'])

In [1]:
import time
import pyaudio
import wave
import speech_recognition as sr
from transformers import pipeline
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Function to record audio
def record_audio_chunk(chunk_duration=6, file_name="buyer_audio.wav"):
    """Records a chunk of audio for the specified duration."""
    chunk = 1024
    sample_format = pyaudio.paInt16
    channels = 1
    rate = 44100

    p = pyaudio.PyAudio()
    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=rate,
                    input=True,
                    frames_per_buffer=chunk)

    print("Listening for the buyer's input...")
    frames = []

    for _ in range(0, int(rate / chunk * chunk_duration)):
        data = stream.read(chunk)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(file_name, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()

    return file_name

# Function to transcribe audio to text
def transcribe_audio(file_name="buyer_audio.wav"):
    """Transcribes audio to text using Google Speech Recognition."""
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_name) as source:
        print("Processing audio transcription...")
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            print(f"Transcription: {text}")
            return text
        except sr.UnknownValueError:
            print("No clear speech detected in the last chunk.")
            return ""
        except sr.RequestError as e:
            print(f"Error in request: {e}")
            return ""

# Function to format customer queries into a conversational structure
def format_customer_inquiry(inquiry):
    formatted_text = """
    Customer Inquiry Summary:
    """
    for i, question in enumerate(inquiry, 1):
        formatted_text += f"{i}. Customer is asking about {question}.\n"
    return formatted_text

# Summarization and sentiment analysis pipeline
def analyze_sentiment_and_summary(chat_history):
    """Analyzes the sentiment and summarizes the chat history in one sentence."""
    # Extract the last line from the chat history
    last_line = chat_history.strip().split('\n')[-1]

    # Summarization pipeline (summarize the last line only)
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(last_line, max_length=20, min_length=5, do_sample=False)
    summary_text = summary[0]['summary_text']
    # Sentiment pipeline
    sentiment_pipeline = pipeline("sentiment-analysis")
    sentiment_result = sentiment_pipeline(summary_text)[0]['label']

    # Map sentiment labels to "Positive" or "Negative"
    sentiment_result = "Positive" if sentiment_result == "POSITIVE" else "Negative"

    # Additional checks for negative sentiment based on indecision or exploring alternatives
    negative_cues = ["check other options", "alternatives", "look at other deals", "explore other options", "not sure", "undecided","check elsewhere"]

    if any(cue in chat_history.lower() for cue in negative_cues):
        sentiment_result = "Negative"
        
    # Determine deal status based on sentiment
    deal_status = "closed" if sentiment_result == "Positive" else "not closed"

    return summary_text, deal_status, sentiment_result

def initialize_google_sheets(credentials_path):
    """Initializes the Google Sheets integration."""
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
    client = gspread.authorize(credentials)
    return client

def append_to_google_sheets(client, spreadsheet_id, row):
    """Appends a row to the specified Google Sheets spreadsheet."""
    sheet = client.open_by_key(spreadsheet_id).sheet1
    sheet.append_row(row)

def main():
    """Main workflow for continuous buyer interaction."""
    credentials_path = "credentials1.json"  # Path to your Google credentials file
    spreadsheet_id = "1Xv7HMn91ddcXhGXeLX-6jxYuVegfgnPIpFIPsZhcvPc"  # Replace with your spreadsheet ID
    sheets_client = initialize_google_sheets(credentials_path)

    print("Starting continuous buyer interaction...")

    buyer_name = input("Please enter the buyer's name: ")
    chat_history = ""
    numbered_inputs = []
    customer_inquiries = []

    while True:
        # Record and transcribe the buyer's audio input
        audio_file = record_audio_chunk()
        transcribed_text = transcribe_audio(audio_file)
        if not transcribed_text:
            print("Waiting for the next buyer input...\n")
            time.sleep(4)  # Pause to wait for the buyer to speak
            continue

        numbered_inputs.append(f"{len(numbered_inputs) + 1}. {transcribed_text}")
        customer_inquiries.append(transcribed_text)  # Add to the customer inquiries
        chat_history += f"{transcribed_text}\n"

        print(f"\nBuyer Transcription: {transcribed_text}")
        print("Waiting for the next interaction...\n")

        # Simulate a button to end chat
        end_chat = input("Type 'end' to finalize the chat or press Enter to continue: ")
        if end_chat.lower() == 'end':
            break

    # Format the input text based on dynamic customer queries
    formatted_text = format_customer_inquiry(customer_inquiries)

    # Summarize the customer inquiry
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(formatted_text, max_length=200, min_length=50, do_sample=False)

    # Print the generated summary
    print("Generated Summary:")
    print(summary[0]['summary_text'])

    # Finalize summary, sentiment, and deal status
    chat_summary, deal_status, sentiment_result = analyze_sentiment_and_summary(chat_history)

    # Append data to Google Sheets including the generated summary
    append_to_google_sheets(sheets_client, spreadsheet_id, [
        buyer_name,
        '\n'.join(numbered_inputs),
        summary[0]['summary_text'],  # Enter the generated summary into the spreadsheet
        sentiment_result,
        deal_status
    ])

    print("Chat session finalized and logged in Google Sheets.")

if __name__ == "__main__":
    main()


Starting continuous buyer interaction...
Listening for the buyer's input...
Processing audio transcription...
No clear speech detected in the last chunk.
Waiting for the next buyer input...

Listening for the buyer's input...
Processing audio transcription...
Transcription: I want some high resolution screen

Buyer Transcription: I want some high resolution screen
Waiting for the next interaction...

Listening for the buyer's input...
Processing audio transcription...
No clear speech detected in the last chunk.
Waiting for the next buyer input...

Listening for the buyer's input...
Processing audio transcription...
Transcription: laptop

Buyer Transcription: laptop
Waiting for the next interaction...





All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.
Device set to use 0
Your max_length is set to 200, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


Generated Summary:
Customer Inquiry Summary: I want some high resolution screen. Customer is asking about laptop. Customer wants to know if there is a way to make the screen bigger. Customer would like to know how much it would cost to buy a new laptop. The customer would also want to know whether there was a way for the customer to make a new screen.


All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.
Device set to use 0
Your max_length is set to 20, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassific

SpreadsheetNotFound: <Response [404]>